In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Recsys2022 Challenge 

NVIDIA-Merlin team participated in [Recsys2022 challenge](http://www.recsyschallenge.com/2022/index.html) and secured 3rd position. This notebook contains the various techniques used in the solution.

### Learning Objective
In this notebook, we will learn the importance of the concepts that improved the results of the competition significantly.

- ##### Label smoothing
     When the probabilities predicted by a Classification model are higher than its accuracy we say the model is overconfident. It can be prevented by using Label smoothing. This technique basically, transforms One-hot encoded labels into smoothed labels. 
$$  \begin{array}{l}
y_{l} \ =\ ( 1\ -\ \alpha \ ) \ *\ y_{o} \ +\ ( \alpha \ /\ L)\\
\alpha :\ Label\ smoothing\\
L:\ Total\ number\ of\ label\ classes\\
y_{o} :\ One-hot\ encoded\ label\ vector
\end{array}
$$
When α is 0, we have the original one-hot encoded labels, and as α increases, we move towards smoothed labels. Read [this](https://arxiv.org/abs/1906.02629) paper to learn more about it.


- ##### Temperature Scaling
    Similar to Label Smoothing, Temperature Scaling is done to reduce the overconfidence of a model. In this, we divide the logits (inputs to the softmax function) by a scalar parameter (T) . For more information on Temperature Scaling read [this](https://arxiv.org/pdf/1706.04599.pdf) paper.
$$ softmax\ =\ \frac{e\ ^{( z_{i} \ /\ \ T)}}{\sum _{j} \ e^{( z_{j} \ /\ T)} \ } $$


- ##### Weight Tying
In this technique, we share the Embedding layer's weights which is used to convert the input to embeddings, as the softmax weights,  to convert hidden layer output to softmax layer output. This drastically reduces the number of parameters and allows the model to train better. For more information read [this](https://arxiv.org/pdf/1608.05859v3.pdf) paper.

In [1]:
import os
import cudf
import dask_cudf
import pandas as pd 

import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.io import Dataset
from merlin.schema import Schema, Tags
from nvtabular.ops import (
    AddMetadata,
)
from merlin.schema.tags import Tags
from utils import get_drepessi_recsys2022_dataset

DATA_FOLDER = 'dressipi'
OUTPUT_FOLDER = 'dressipi_processed'
DATETIME_CONVERTION = 'ms'

## Dressipi
The [Dressipi](http://www.recsyschallenge.com/2022/dataset.html) dataset contains 1.1 M online retail sessions that resulted in a purchase. It provides details about items that were viewed in a session, the item purchased at the end of the session and numerous features of those items. The task of this competition was, given a sequence of items predict which item will be purchased at the end of a session.


<img src="images/dressipi.jpeg" alt="dressipi_dataset" style="width: 400px; float: center;">  


In [2]:
train, valid, item_features, sessions = get_drepessi_recsys2022_dataset(DATA_FOLDER)

## Feature Engineering with NVTabular

### Categorify

In [3]:
item_features_names = item_features.columns[1:].tolist()
cat_features = ['session_id', 'item_id'] + item_features_names >> nvt.ops.Categorify()
# all_data = dask_cudf.concat([Dataset(sessions).to_ddf()])
all_data = Dataset(sessions).to_ddf()

features = ['timestamp','date'] + cat_features
dataset = Dataset(all_data)
workflow0 = nvt.Workflow(features)
workflow0.fit(dataset)

# transform data
train_0 = workflow0.transform(Dataset(train))
valid_0 = workflow0.transform(Dataset(valid))

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


### GroupBy

In [4]:
%%time
features = train_0.head().columns.tolist()

# Define Groupby Operator
to_aggregate = {
    'date': ["first", "last"],
    'item_id': ["list", "first", "last"],
    'timestamp': ["list"]
}
for name in item_features_names: 
    to_aggregate[name] = ['list']
    
groupby_features = features >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    sort_cols=["date"],
    aggs= to_aggregate,
    name_sep="_")

# Add tags needed for the t4rec models definition
item_list = groupby_features['item_id_list'] >> nvt.ops.AddMetadata(tags=[Tags.SEQUENCE, Tags.ITEM, Tags.ITEM_ID, Tags.LIST])
item_first_last = groupby_features['item_id_last', 'item_id_first'] >> nvt.ops.AddMetadata(tags=[Tags.ITEM, Tags.ITEM_ID])
feature_list = groupby_features[[name+'_list' for name in item_features_names]]>> nvt.ops.AddMetadata(tags=[Tags.SEQUENCE, Tags.ITEM, Tags.LIST])
other_features = groupby_features['session_id', 'date_first', 'date_last','timestamp_list']

workflow1 = nvt.Workflow(item_list + item_first_last + feature_list + other_features)
workflow1.fit(train_0)

# transform data
train_1 = workflow1.transform(train_0)
valid_1 = workflow1.transform(valid_0)

CPU times: user 1.42 s, sys: 1.74 s, total: 3.17 s
Wall time: 4.7 s


### Truncate and Padding for a Maximum Sequence Length

In [5]:
SESSIONS_MAX_LENGTH = 20
list_cols = [col for col in train_1.head().columns if 'list' in col and 'date' not in col]
truncated_fatures = list_cols >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH, pad=True) >> nvt.ops.Rename(postfix = '_seq')
final_features = [
    'session_id', 'date_first', 'date_last', 'item_id_list', 'item_id_first', 
]

### Add TARGET tag in label column

In [6]:
LABEL_COLUMNS = ['item_id_last']
label_features = LABEL_COLUMNS >> AddMetadata(tags=[str(Tags.BINARY_CLASSIFICATION), "target"])

In [7]:
workflow2 = nvt.Workflow(final_features + truncated_fatures+label_features)
workflow2.fit(train_1)

# transform data
train_2 = workflow2.transform(train_1)
valid_2 = workflow2.transform(valid_1)

### Save processed data to Parquet files

In [8]:
%%time
train_ds = Dataset(train_2.to_ddf().sort_values('date_last'), schema=train_2.schema)
valid_ds = Dataset(valid_2.to_ddf().sort_values('date_last'), schema=valid_2.schema)

train_ds.to_parquet(os.path.join(OUTPUT_FOLDER, "train/"), output_files=10)
valid_ds.to_parquet(os.path.join(OUTPUT_FOLDER, "valid/"), output_files=10)

CPU times: user 21.3 s, sys: 22.1 s, total: 43.4 s
Wall time: 1min 11s


## Training - MLP

A sequential-MLP model with average of the sequence as final representation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import os
import cudf
import pandas as pd 
import tensorflow as tf
from merlin.io import Dataset
from merlin.schema import Tags
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from merlin.models.tf.dataset import BatchedDataset
from merlin.models.tf.utils.tf_utils import extract_topk
import numpy as np
import merlin.models.tf as mm
from merlin.models.tf import InputBlock
from merlin.models.tf.models.base import Model
from merlin.models.tf.core.aggregation import SequenceAggregation, SequenceAggregator
from merlin.models.tf.core.transformations import (
    ItemsPredictionWeightTying,
    L2Norm,
    LogitsTemperatureScaler,
)

DATA_FOLDER = 'dressipi'
DATA_PROCESSED_FOLDER = 'dressipi_processed'

2022-07-14 04:54:53.194106: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-14 04:54:55.175395: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16255 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:07:00.0, compute capability: 7.0


In [3]:
train = Dataset(os.path.join(DATA_PROCESSED_FOLDER, 'train/*.parquet'),shuffle=False)
valid = Dataset(os.path.join(DATA_PROCESSED_FOLDER, 'valid/*.parquet'), shuffle=False)

purchases = pd.read_csv(os.path.join(DATA_FOLDER, "train_purchases.csv"))
item_map = pd.read_parquet(
    os.path.join("categories", "unique.item_id.parquet"))['item_id'].to_dict()
session_map = pd.read_parquet(
    os.path.join("categories", "unique.session_id.parquet"))['session_id'].to_dict()

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [4]:
schema_model = train.schema.select_by_name(['item_id_list_seq', 'item_id_last'])
schema_model

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,item_id_list_seq,"(Tags.CATEGORICAL, Tags.SEQUENCE, Tags.ITEM_ID...",int64,True,False,None,0.0,0.0,0.0,.//categories/unique.item_id.parquet,23497.0,449.0,0,23497,item_id
1,item_id_last,"(Tags.CATEGORICAL, Tags.TARGET, Tags.BINARY_CL...",int64,False,False,None,0.0,0.0,0.0,.//categories/unique.item_id.parquet,23497.0,449.0,0,23497,item_id


In [5]:
import datetime

tmp = train.compute()
start_month_train = tmp[tmp['date_last'] <= datetime.datetime(2021, 4, 1)]
last_month_train = tmp[tmp['date_first'] >= datetime.datetime(2021, 4, 1)]
start_month_train =  Dataset(start_month_train, schema=schema_model, shuffle=True)
last_month_train =  Dataset(last_month_train, schema=schema_model, shuffle=True)

In [6]:
tmp.shape,start_month_train.compute().shape,last_month_train.compute().shape

((920830, 30), (848637, 30), (72193, 30))

In [7]:
EPOCHS = 2
BATCH_SIZE = 128

### Model
InputBlock which takes sequential features, concatenate them and return the sequence of interaction embeddings

In [8]:
input_block = InputBlock(
        schema_model,
        aggregation='concat',
        seq=True,
        max_seq_length=20,
        embedding_options=mm.EmbeddingOptions(embedding_dim_default=128),
        split_sparse=True,
)

MLPBlock to get the sequence of hidden representation

In [9]:
mlp_block = mm.MLPBlock(
                [64, 128],
                activation='relu',
                no_activation_last_layer=True,
                dropout=0.01,
            )

Multi-Classiffication Prediction head which has
- Layer Normalization
- Weight Tying
- Labels as One-hot encoded vectors, used for label smoothing 
- Temperature Scaling to reduce the overconfidence of the model

In [10]:
prediction_call = L2Norm().connect(
    ItemsPredictionWeightTying(schema_model), 
    mm.LabelToOneHot(), 
    LogitsTemperatureScaler(temperature=2)
)

prediction_task = mm.MultiClassClassificationTask(
    target_name="item_id_last",
    pre=prediction_call,
)

Now, we connect all the blocks togther to build a model

In [11]:
model_mlp = Model(input_block, mlp_block, SequenceAggregator(SequenceAggregation.MEAN), prediction_task)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=3e-1,
    clipnorm=True
)

# model_mlp.compile(optimizer=optimizer, run_eagerly=False)
model_mlp.compile(
    optimizer=optimizer,
    run_eagerly=True,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.2),
    metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[100])
)

2022-07-14 04:55:02.477037: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


### Model Training

In [12]:
%%time
history = model_mlp.fit(start_month_train, validation_data=last_month_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/2
6630/6630 [==============================] - 430s 63ms/step - loss: 6.6747 - recall_at_100: 0.5320 - mrr_at_100: 0.2474 - ndcg_at_100: 0.3055 - map_at_100: 0.2474 - precision_at_100: 0.0053 - regularization_loss: 0.0000e+00 - val_loss: 6.9967 - val_recall_at_100: 0.6474 - val_mrr_at_100: 0.3363 - val_ndcg_at_100: 0.4019 - val_map_at_100: 0.3363 - val_precision_at_100: 0.0065 - val_regularization_loss: 0.0000e+00
Epoch 2/2
6630/6630 [==============================] - 419s 62ms/step - loss: 5.7599 - recall_at_100: 0.8505 - mrr_at_100: 0.4635 - ndcg_at_100: 0.5476 - map_at_100: 0.4635 - precision_at_100: 0.0085 - regularization_loss: 0.0000e+00 - val_loss: 6.8384 - val_recall_at_100: 0.6827 - val_mrr_at_100: 0.3590 - val_ndcg_at_100: 0.4280 - val_map_at_100: 0.3590 - val_precision_at_100: 0.0068 - val_regularization_loss: 0.0000e+00
CPU times: user 15min 7s, sys: 23.2 s, total: 15min 30s
Wall time: 14min 21s


In [13]:
def compute_mrr(rec_list,target):
    mrr = 0
    for a,b in zip(rec_list,target):
        rank = np.argmax(np.array(a)==b)
        if rank != 0:
            mrr += (1 / (1 + rank))
    return mrr/(target.shape[0])

In [14]:
def model_evaluation(model):
    valid = Dataset(
    [DATA_PROCESSED_FOLDER+'/valid/*.parquet'], 
    part_mem_fraction=0.01, 
    shuffle=False)
    x = BatchedDataset(
        valid, 
        batch_size=256, 
        shuffle=False, 
    )
    predictions = model.predict(x)

    topk_predicted = []
    for i in range(predictions.shape[0]):
        _, topk_indices = tf.math.top_k(predictions[i, :], 100)
        topk_predicted.append(topk_indices.numpy().reshape(1, 100))

    top_predicted = np.concatenate(topk_predicted)

    valid_data = valid.to_ddf().compute().to_pandas()
    valid_data['session_id'] = valid_data.session_id.map(session_map)
    valid_data = pd.merge(valid_data, purchases, on='session_id')[['session_id', 'item_id']]

    valid_data['top100_predicted'] = top_predicted.tolist()
    valid_data['top100_predicted']= valid_data['top100_predicted'].apply(lambda x: [item_map[i] for i in x])

    return compute_mrr(valid_data['top100_predicted'], valid_data['item_id'])

### Model Evaluation

In [15]:
%%time
model_evaluation(model_mlp)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(
/tmp/ipykernel_91629/1660147556.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  rank = np.argmax(np.array(a)==b)


CPU times: user 2min 12s, sys: 46.1 s, total: 2min 58s
Wall time: 2min 26s


0.07580546350707382

## Training Bi-LSTM

In [16]:
EPOCHS = 1
BATCH_SIZE = 128
BI_LSTM_HIDDEN_DIM = 64

### Model

BiLSTM Block: It requires a dictionary input with the sequence of interaction embeddings `input_sequence`

In [17]:
class BiLSTM(mm.Block):
    def __init__(self, hidden_dim= 64, **kwargs):
        self.hidden_dim = hidden_dim
        lstm = tf.keras.layers.LSTM(hidden_dim, return_sequences=False, dropout=0.05,
                                   kernel_regularizer=regularizers.l2(1e-4))
        self.lstm = tf.keras.layers.Bidirectional(lstm)
        
        super().__init__(**kwargs)
        
    def call(self, inputs, training=False, **kwargs) -> tf.Tensor:  
        interactions = inputs['input_sequence']
        sequence_representation = self.lstm(interactions)
        return sequence_representation
    
    def compute_output_shape(self, input_shape):
        input_shape = input_shape['input_sequence']
        return (input_shape[0], input_shape[1], self.hidden_dim*2)
    
    
bilstm = BiLSTM(hidden_dim=BI_LSTM_HIDDEN_DIM)

InputBlock which takes sequential features, concatenate them and return the sequence of interaction embeddings

In [18]:
inputs = InputBlock(
        schema_model,
        aggregation='concat',
        seq=True,
        max_seq_length=20,
        embedding_options=mm.EmbeddingOptions(
            embedding_dim_default=128,
            infer_embedding_sizes=True,
            infer_embedding_sizes_multiplier=2,
            infer_embeddings_ensure_dim_multiple_of_8=True
        ),
        split_sparse=True,
)

In [19]:
dense_block = mm.ParallelBlock({'input_sequence': inputs}).connect(bilstm)

MLPBlock to get the sequence of hidden representation

In [20]:
mlp_block = mm.MLPBlock(
                [64, 32],
                activation='relu',
                no_activation_last_layer=True,
                dropout=0.01,
            )

Multi-Classiffication Prediction head which has
- Layer Normalization
- Weight Tying
- Labels as One-hot encoded vectors, used for label smoothing 
- Temperature Scaling to reduce the overconfidence of the model

In [21]:
prediction_call = L2Norm().connect(
    ItemsPredictionWeightTying(schema_model), 
    mm.LabelToOneHot(), 
    LogitsTemperatureScaler(temperature=2)
)

prediction_task = mm.MultiClassClassificationTask(
    target_name="item_id_last",
    pre=prediction_call,
)

Now, we connect all the blocks togther to build a model

In [22]:
model_bi_lstm = Model(dense_block, mlp_block, prediction_task)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=3e-1,
    clipnorm=True
)

model_bi_lstm.compile(
    optimizer=optimizer,
    run_eagerly=True,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.2),
    metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[100])
)

### Model Training

In [23]:
%%time
history = model_bi_lstm.fit(start_month_train, validation_data=last_month_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

2022-07-14 05:33:16.035175: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400


6630/6630 [==============================] - 681s 101ms/step - loss: 25.8208 - recall_at_100: 0.2871 - mrr_at_100: 0.0584 - ndcg_at_100: 0.1004 - map_at_100: 0.0584 - precision_at_100: 0.0029 - regularization_loss: 17.2444 - val_loss: 27.7185 - val_recall_at_100: 0.2704 - val_mrr_at_100: 0.0753 - val_ndcg_at_100: 0.1120 - val_map_at_100: 0.0753 - val_precision_at_100: 0.0027 - val_regularization_loss: 18.6344
CPU times: user 11min 58s, sys: 13.8 s, total: 12min 12s
Wall time: 11min 31s


### Model Evaluation

In [24]:
%%time
model_evaluation(model_bi_lstm)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(
/tmp/ipykernel_91629/1660147556.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  rank = np.argmax(np.array(a)==b)


CPU times: user 2min 15s, sys: 55.5 s, total: 3min 10s
Wall time: 2min 36s


0.017758847633143584